In [1]:
import csv

def read_csv_to_dict(file_path):
    data = {}
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if len(row) == 2:  # Ensure there are exactly 2 elements in the row
                key, value = row
                data[key] = int(value)  # Convert the value to integer
    return data

if __name__ == "__main__":
    csv_file_path = r'D:\IdeaProjects\IITP_synth\lv-parametric-modelling\ipynb\edinburgh\ICV_scan1_1_5T.csv'
    
    # Read the CSV file and create the dictionary
    data_dict = read_csv_to_dict(csv_file_path)
    
    # Print the resulting dictionary
    print("Data Dictionary:", data_dict)
    total = sum(data_dict.values())
    count = len(data_dict)
    etiv_mean = total/count
    print("Total:", total)
    print("Count:", count)
    print("Average:", total / count)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
Data Dictionary: {'360002': 1560774, '360003': 1339630, '360004': 1282520, '360006': 1438164, '360007': 1584316, '360008': 1639524, '360009': 1383426, '360013': 1304950, '360015': 1460480, '360017': 1514154, '360020': 1389564, '360021': 1354876, '360022': 1371402, '360023': 1312022, '360026': 1447958, '360029': 1571690, '360030': 1417604, '360031': 1453832, '360032': 1465240, '360034': 1376432, '360036': 1405024, '360038': 1743896, '360040': 1442262, '360041': 1628892, '360042': 1506414, '360043': 1612300, '360045': 1217336, '360048': 1408820, '360051': 1606914, '360052': 1286936, '360055': 1277670, '360056': 1536788, '360058': 1478234, '360059': 1296008, '360061': 1585658, '360063': 1369666, '360066': 1458810, '360067': 1561278, '360069': 1394312, '360070': 1445700, '360072': 1388640, '360074': 1587772, '360076': 1294366, '360079': 1457058,

In [2]:
########## Import Libararies
import os
import pickle
import glob
from tqdm.notebook import tqdm

import numpy as np
from skimage import measure
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

import open3d as o3d
from compas.geometry import trimesh_remesh
from compas.datastructures import Mesh
import nibabel as nib

from util import *

import csv
import copy

flag = "manual" # manual or synthseg

import csv
def read_sub(file_path):
    data = {}
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        for index, row in enumerate(csv_reader):
            if index==0: continue
            else:
                id= row[0]
                data[row[0]]=row[1:6]
    return data
common_sub_csv =  rf"D:\NextCloud\원정\2024\Data-to-KAIST\sub.csv"
common_dict = read_sub(common_sub_csv)
print(len(common_dict), common_dict)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
693 {'LBC360002': ['1', '1', '1', '1', '1'], 'LBC360003': ['1', '1', '0', '0', '0'], 'LBC360004': ['1', '1', '1', '1', '1'], 'LBC360006': ['1', '1', '1', '1', '1'], 'LBC360007': ['1', '1', '1', '1', '1'], 'LBC360008': ['1', '1', '1', '1', '1'], 'LBC360009': ['1', '1', '1', '1', '0'], 'LBC360013': ['1', '0', '0', '0', '0'], 'LBC360015': ['1', '1', '1', '1', '1'], 'LBC360017': ['1', '1', '1', '1', '0'], 'LBC360020': ['1', '1', '0', '1', '1'], 'LBC360021': ['1', '1', '1', '0', '0'], 'LBC360022': ['1', '1', '1', '1', '0'], 'LBC360023': ['1', '0', '1', '1', '1'], 'LBC360025': ['0', '0', '1', '1', '1'], 'LBC360026': ['1', '1', '1', '1', '0'], 'LBC360029': ['1', '0', '0', '0', '0'], 'LBC360030': ['1', '1', '1', '1', '1'], 'LBC360031': ['1', '1', '1', '1', '1'], 'LBC360032': ['1', '1', '0', '0', '0'], 'LBC360034': ['1', '1', '1

# Longi deformity viewer

In [ ]:
classes = [10, 11, 12, 13, 17, 18, 26, 28, 49, 50, 51, 52, 53, 54, 58, 60]
for label in classes:
	temp_mesh = o3d.io.read_triangle_mesh(rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\whole_brain_structure\temp_meshes\{label}_temp.obj")
	temp_mesh.compute_vertex_normals()
	temp_mesh.paint_uniform_color([0.7, 1.0, 0.7])

	count=0
	temp_vert_shape = np.asarray(temp_mesh.vertices).shape
	colors = np.zeros((693,*temp_vert_shape))
	print(colors.shape)

	temp_mesh_draw = copy.deepcopy(temp_mesh)

	t1, t2 = (0,4)
	count=0
	for index, key in enumerate(common_dict.keys()):
		if common_dict[key][t1] == '1' and common_dict[key][t2] == '1' and key[3:] in data_dict.keys():
			past = o3d.utility.Vector3dVector(np.load(glob.glob(rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan{t1+1}_optimed_data\{label}_scan{t1+1}\out\{key}\5000*.npy")[0])[0])
			now = o3d.utility.Vector3dVector(np.load(glob.glob(rf"D:\Data-to-KAIST\Task4) longi-subcortical\scan{t2+1}_optimed_data\{label}_scan{t2+1}\out\{key}\5000*.npy")[0])[0])
			delta = np.array(now)-np.array(past)
			norm = np.linalg.norm(delta,axis=1)

			print(norm.shape)
			# Get the viridis colormap
			viridis = plt.cm.viridis

			# Map the normalized value to a color
			color = np.asarray(viridis(norm)) # Euclidean distance from mean
			print(np.asarray(color).shape)
			print(color[:5])
			temp_mesh.vertex_colors = o3d.utility.Vector3dVector(color[:,:3])
			colors[count] =color[:,:3]
			count+=1
			temp_mesh.vertices = past
			temp_mesh.compute_vertex_normals()
			new_norm = np.asarray(temp_mesh.vertices)
			print(new_norm.shape, delta.shape)
			dot_products = np.einsum('ij,ij->i', new_norm, delta)
			dot_products[dot_products<0] = -1
			dot_products[dot_products>=0] = 1
			disp = dot_products * norm # only x direction
			etiv = int(data_dict[key[3:]])
			norm_factor = etiv/etiv_mean
			# disp = disp /norm_factor
			# now_temp_mesh = copy.deepcopy(temp_mesh)
			# now_new_vert= np.asarray(now).copy()
			# now_new_vert[:,0]+=20
			# now_temp_mesh.vertices = o3d.utility.Vector3dVector(now_new_vert)
			
			# o3d.visualization.draw_geometries([temp_mesh, now_temp_mesh])
			# if index>2:break

	print(count)
	new_colors = colors[:count]
	new_colors_mean = new_colors.mean(axis=0)
	print(new_colors_mean.shape)

	temp_mesh_draw.vertex_colors = o3d.utility.Vector3dVector(np.asarray(new_colors_mean))

	o3d.visualization.draw_geometries([temp_mesh_draw])



(693, 1412, 3)


IndexError: list index out of range

# File generator

In [94]:
L_temp_mesh = o3d.io.read_triangle_mesh(r"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\data\hippo\new_temp_mesh_L_longi.obj")
L_temp_mesh.compute_vertex_normals()
L_temp_mesh.paint_uniform_color([0.7, 1.0, 0.7])
R_temp_mesh = o3d.io.read_triangle_mesh(r"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\data\hippo\new_temp_mesh_R_longi.obj")
R_temp_mesh.compute_vertex_normals()
R_temp_mesh.paint_uniform_color([1.0, 0.7, 0.7])
o3d.visualization.draw_geometries([L_temp_mesh, R_temp_mesh])
count=0
colors = np.zeros((693,2587,3))
mode = "r"

if mode =="l":
    temp_mesh = L_temp_mesh
else:
	temp_mesh = R_temp_mesh

past_now_pairs = [(0,1),(0,2),(0,4),(1,2),(2,3),(2,4),(3,4)]
for pair in past_now_pairs:
	t1, t2 = pair
	count=0
	with open(f'LocalDeformity_longi_{t1+1}{t2+1}_R.txt', 'a') as f:
		for index, key in enumerate(common_dict.keys()):
			if common_dict[key][t1] == '1' and common_dict[key][t2] == '1' and key[3:] in data_dict.keys():
				past = o3d.utility.Vector3dVector(np.load(glob.glob(rf"D:\Data-to-KAIST\Task3) longi-hippo\scan{t1+1}\{mode}\out\{key}\4000*.npy")[0])[0])
				now = o3d.utility.Vector3dVector(np.load(glob.glob(rf"D:\Data-to-KAIST\Task3) longi-hippo\scan{t2+1}\{mode}\out\{key}\4000*.npy")[0])[0])
				delta = np.array(now)-np.array(past)
				norm = np.linalg.norm(delta,axis=1)

				print(norm.shape)
				# Get the viridis colormap
				viridis = plt.cm.viridis

				# Map the normalized value to a color
				color = np.asarray(viridis(norm)) # Euclidean distance from mean
				print(np.asarray(color).shape)
				print(color[:5])
				temp_mesh.vertex_colors = o3d.utility.Vector3dVector(color[:,:3])
				colors[count] =color[:,:3]
				count+=1
				temp_mesh.vertices = past
				temp_mesh.compute_vertex_normals()
				new_norm = np.asarray(temp_mesh.vertices)
				print(new_norm.shape, delta.shape)
				dot_products = np.einsum('ij,ij->i', new_norm, delta)
				dot_products[dot_products<0] = -1
				dot_products[dot_products>=0] = 1
				disp = dot_products * norm # only x direction
				etiv = int(data_dict[key[3:]])
				norm_factor = etiv/etiv_mean
				# disp = disp /norm_factor
				sub_id = key[3:]+"_R"
				f.write(f"{sub_id} ")
				np.savetxt(f, disp, fmt='%.5f', delimiter=' ', newline=' ')
				f.write(f" ***")
				
				# now_temp_mesh = copy.deepcopy(temp_mesh)
				# now_new_vert= np.asarray(now).copy()
				# now_new_vert[:,0]+=20
				# now_temp_mesh.vertices = o3d.utility.Vector3dVector(now_new_vert)
				
				# o3d.visualization.draw_geometries([temp_mesh, now_temp_mesh])
				# if index>2:break
  
print(count)
new_colors = colors[:count]
new_colors_mean = new_colors.mean(axis=0)
print(new_colors_mean.shape)

temp_mesh.vertex_colors = o3d.utility.Vector3dVector(np.asarray(new_colors_mean))

o3d.visualization.draw_geometries([temp_mesh])
  
	

(2587,)
(2587, 4)
[[0.149039 0.508051 0.55725  1.      ]
 [0.165117 0.467423 0.558141 1.      ]
 [0.128729 0.563265 0.551229 1.      ]
 [0.133743 0.548535 0.553541 1.      ]
 [0.136408 0.541173 0.554483 1.      ]]
(2587, 3) (2587, 3)
(2587,)
(2587, 4)
[[0.252899 0.742211 0.448284 1.      ]
 [0.202219 0.715272 0.476084 1.      ]
 [0.153894 0.680203 0.504172 1.      ]
 [0.132444 0.552216 0.553018 1.      ]
 [0.120565 0.596422 0.543611 1.      ]]
(2587, 3) (2587, 3)
(2587,)
(2587, 4)
[[0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]]
(2587, 3) (2587, 3)
(2587,)
(2587, 4)
[[0.395174 0.797475 0.367757 1.      ]
 [0.344074 0.780029 0.397381 1.      ]
 [0.783315 0.879285 0.125405 1.      ]
 [0.487026 0.823929 0.312321 1.      ]
 [0.575563 0.844566 0.256415 1.      ]]
(2587, 3) (2587, 3)
(2587,)
(2587, 4)
[[0.151918 0.500685 0.557587 1.      ]
 [0.149

# Diff data generator